In [1]:
import numpy as np
import pandas as pd


%time
import os
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import sklearn

# Libraries and packages for text (pre-)processing 
import string
import re
import nltk


CPU times: total: 0 ns
Wall time: 0 ns


In [2]:
df_train = pd.read_csv('train_clean.csv')
df_test = pd.read_csv('test_clean.csv')

In [3]:
df_train_wo = pd.read_csv('train_clean_wo.csv')
df_test_wo = pd.read_csv('test_clean_wo.csv')

In [4]:
X_train, y_train, X_test,y_test = df_train['lemmatize_text'], df_train['Depression'].astype('int'), df_test['lemmatize_text'], df_test['Depression'].astype('int')

In [5]:
X_train_wo, y_train_wo, X_test_wo,y_test_wo = df_train_wo['lemmatize_text_wo'], df_train_wo['Depression'].astype('int'), df_test_wo['lemmatize_text_wo'], df_test_wo['Depression'].astype('int')

In [9]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
import sklearn.metrics as metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
def get_models():
    models = dict()
    #models['lr'] = LogisticRegression()
    #models['knn'] = KNeighborsClassifier()
    #models['svm'] = LinearSVC(random_state=0, tol=1e-5)
    #models['RF']=RandomForestClassifier(criterion='gini')
    models['Ada']=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,n_estimators=100, random_state=None)
    models['XGB']=XGBClassifier()
    models['catboost'] = CatBoostClassifier(iterations=5, learning_rate=0.1)
    models['cat'] = CatBoostClassifier(
        random_seed = 42,
        logging_level = 'Silent'
    )
    models['cat2'] = CatBoostClassifier(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
    models['light'] = lgb.LGBMClassifier()

    return models

def get_models1():
    models['light'] = lgb.LGBMClassifier()
    return models


def get_models2():
    models = dict()
    models['cat2'] = CatBoostClassifier(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
    return models

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

def eval_confusion(y_pred, y_true):
    conf_matrix = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {'matrix': conf_matrix, 'precision': precision, 'recall': recall, 'f1': f1}

## Weight Words(5000)


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

def cv(data, ngram = 1, MAX_NB_WORDS = 5000):
    count_vectorizer = CountVectorizer(ngram_range = (ngram, ngram), max_features = MAX_NB_WORDS)
    emb = count_vectorizer.fit_transform(data).toarray()
    print("count vectorize with", str(np.array(emb).shape[1]), "features")
    return emb, count_vectorizer

In [11]:
def print_out(emb, feat, ngram, compared_sentence=0):
    print(ngram,"bag-of-words: ")
    print(feat.get_feature_names(), "\n")
    print(ngram,"bag-of-feature: ")
    print(test_cv_1gram.vocabulary_, "\n")
    print("BoW matrix:")
    print(pd.DataFrame(emb.transpose(), index = feat.get_feature_names()).head(), "\n")
    print(ngram,"vector example:")
    print(df_train["lemmatize_text"][compared_sentence])
    print(emb[compared_sentence], "\n")

In [12]:
%time 

# implement into the whole dataset
train_df_corpus = df_train["lemmatize_text"].tolist()
train_df_em_1gram, vc_1gram = cv(train_df_corpus, 1)
train_df_em_2gram, vc_2gram = cv(train_df_corpus, 2)
train_df_em_3gram, vc_3gram = cv(train_df_corpus, 3)

print(len(train_df_corpus))
print(train_df_em_1gram.shape)
print(train_df_em_2gram.shape)
print(train_df_em_3gram.shape)

CPU times: total: 0 ns
Wall time: 0 ns
count vectorize with 5000 features
count vectorize with 5000 features
count vectorize with 5000 features
887
(887, 5000)
(887, 5000)
(887, 5000)


In [13]:

# implement into the whole dataset
test_df_corpus = df_test["lemmatize_text"].tolist()
test_df_em_1gram, vc_1gram = cv(test_df_corpus, 1)
test_df_em_2gram, vc_2gram = cv(test_df_corpus, 2)
test_df_em_3gram, vc_3gram = cv(test_df_corpus, 3)

print(len(test_df_corpus))
print(test_df_em_1gram.shape)
print(test_df_em_2gram.shape)
print(test_df_em_2gram.shape)

count vectorize with 5000 features
count vectorize with 5000 features
count vectorize with 5000 features
820
(820, 5000)
(820, 5000)
(820, 5000)


In [14]:
# evaluate each model on the training set  (5000)

for clf_name, clf in get_models().items():
    clf.fit(train_df_em_2gram, y_train)
    y_pred = clf.predict(test_df_em_2gram)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[655  86]
 [ 72   7]]
precision: 0.07526881720430108
recall: 0.08860759493670886
f1-score: 0.08139534883720931
accuracy: 0.8073170731707318

XGB:
[[555 186]
 [ 52  27]]
precision: 0.1267605633802817
recall: 0.34177215189873417
f1-score: 0.18493150684931506
accuracy: 0.7097560975609756

0:	learn: 0.6344558	total: 187ms	remaining: 749ms
1:	learn: 0.5896761	total: 210ms	remaining: 315ms
2:	learn: 0.5524731	total: 236ms	remaining: 157ms
3:	learn: 0.5223747	total: 259ms	remaining: 64.8ms
4:	learn: 0.5003254	total: 282ms	remaining: 0us
catboost:
[[740   1]
 [ 78   1]]
precision: 0.5
recall: 0.012658227848101266
f1-score: 0.02469135802469136
accuracy: 0.9036585365853659

cat:
[[694  47]
 [ 69  10]]
precision: 0.17543859649122806
recall: 0.12658227848101267
f1-score: 0.14705882352941177
accuracy: 0.8585365853658536

0:	learn: 0.6618990	total: 96.5ms	remaining: 48.2s
50:	learn: 0.2702331	total: 4.77s	remaining: 42s
100:	learn: 0.1672634	total: 9.84s	remaining: 38.9s
150:	learn: 0.1005536	

In [15]:
# evaluate each model on the training set  (5000)  3 gram

for clf_name, clf in get_models().items():
    clf.fit(train_df_em_3gram, y_train)
    y_pred = clf.predict(test_df_em_3gram)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    #print (metrics.classification_report (y_test, y_pred, 
     #       target_names = ['0','1']))
    print()

Ada:
[[623 118]
 [ 66  13]]
precision: 0.09923664122137404
recall: 0.16455696202531644
f1-score: 0.1238095238095238
accuracy: 0.775609756097561

XGB:
[[663  78]
 [ 70   9]]
precision: 0.10344827586206896
recall: 0.11392405063291139
f1-score: 0.10843373493975904
accuracy: 0.8195121951219512

0:	learn: 0.6369178	total: 13.3ms	remaining: 53.3ms
1:	learn: 0.5887500	total: 28.9ms	remaining: 43.4ms
2:	learn: 0.5540855	total: 43ms	remaining: 28.7ms
3:	learn: 0.5264174	total: 57ms	remaining: 14.2ms
4:	learn: 0.5039144	total: 73.4ms	remaining: 0us
catboost:
[[741   0]
 [ 79   0]]
precision: 0.0
recall: 0.0
f1-score: 0.0
accuracy: 0.9036585365853659

cat:
[[729  12]
 [ 79   0]]
precision: 0.0
recall: 0.0
f1-score: 0.0
accuracy: 0.8890243902439025

0:	learn: 0.6638540	total: 73.6ms	remaining: 36.7s
50:	learn: 0.3408075	total: 3.46s	remaining: 30.5s
100:	learn: 0.2824800	total: 7.2s	remaining: 28.4s
150:	learn: 0.2465532	total: 10.9s	remaining: 25.2s
200:	learn: 0.1962529	total: 14.8s	remaining: 2

# TF-IDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

def TFIDF(data, ngram = 1, MAX_NB_WORDS = 5000):
    tfidf_x = TfidfVectorizer(ngram_range = (ngram, ngram), max_features = MAX_NB_WORDS)
    emb = tfidf_x.fit_transform(data).toarray()
    print("tf-idf with", str(np.array(emb).shape[1]), "features")
    return emb, tfidf_x

In [18]:

# implement into the whole dataset
train_df_corpus = df_train["lemmatize_text"].tolist()
train_df_tfidf_1gram, tfidf_1gram = TFIDF(train_df_corpus, 1)
train_df_tfidf_2gram, tfidf_2gram = TFIDF(train_df_corpus, 2)
train_df_tfidf_3gram, tfidf_3gram = TFIDF(train_df_corpus, 3)

print(len(train_df_corpus))
print(train_df_tfidf_1gram.shape)
print(train_df_tfidf_1gram.shape)
print(train_df_tfidf_1gram.shape)

tf-idf with 5000 features
tf-idf with 5000 features
tf-idf with 5000 features
887
(887, 5000)
(887, 5000)
(887, 5000)


In [19]:
test_df_corpus = df_test["lemmatize_text"].tolist()
test_df_tfidf_1gram, tfidf_1gram = TFIDF(test_df_corpus, 1)
test_df_tfidf_2gram, tfidf_2gram = TFIDF(test_df_corpus, 2)
test_df_tfidf_3gram, tfidf_3gram = TFIDF(test_df_corpus, 3)

print(len(test_df_corpus))
print(test_df_tfidf_1gram.shape)

tf-idf with 5000 features
tf-idf with 5000 features
tf-idf with 5000 features
820
(820, 5000)


In [20]:
for clf_name, clf in get_models().items():
    clf.fit(train_df_tfidf_3gram, y_train)
    y_pred = clf.predict(test_df_tfidf_3gram)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[693  48]
 [ 69  10]]
precision: 0.1724137931034483
recall: 0.12658227848101267
f1-score: 0.145985401459854
accuracy: 0.8573170731707317

XGB:
[[710  31]
 [ 77   2]]
precision: 0.06060606060606061
recall: 0.02531645569620253
f1-score: 0.03571428571428571
accuracy: 0.8682926829268293

0:	learn: 0.6369175	total: 25.7ms	remaining: 103ms
1:	learn: 0.5917249	total: 58.1ms	remaining: 87.2ms
2:	learn: 0.5539013	total: 87.5ms	remaining: 58.4ms
3:	learn: 0.5243155	total: 118ms	remaining: 29.5ms
4:	learn: 0.5014192	total: 152ms	remaining: 0us
catboost:
[[741   0]
 [ 79   0]]
precision: 0.0
recall: 0.0
f1-score: 0.0
accuracy: 0.9036585365853659

cat:
[[738   3]
 [ 79   0]]
precision: 0.0
recall: 0.0
f1-score: 0.0
accuracy: 0.9

0:	learn: 0.6616078	total: 264ms	remaining: 2m 11s
50:	learn: 0.2940723	total: 14.6s	remaining: 2m 8s
100:	learn: 0.2218478	total: 29.1s	remaining: 1m 54s
150:	learn: 0.1708375	total: 43.3s	remaining: 1m 40s
200:	learn: 0.1181414	total: 57.5s	remaining: 1m 25s
250:	l

In [21]:
for clf_name, clf in get_models().items():
    clf.fit(train_df_tfidf_2gram, y_train)
    y_pred = clf.predict(test_df_tfidf_2gram)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[687  54]
 [ 70   9]]
precision: 0.14285714285714285
recall: 0.11392405063291139
f1-score: 0.1267605633802817
accuracy: 0.848780487804878

XGB:
[[739   2]
 [ 78   1]]
precision: 0.3333333333333333
recall: 0.012658227848101266
f1-score: 0.024390243902439022
accuracy: 0.9024390243902439

0:	learn: 0.6331178	total: 127ms	remaining: 510ms
1:	learn: 0.5840649	total: 245ms	remaining: 367ms
2:	learn: 0.5423852	total: 364ms	remaining: 242ms
3:	learn: 0.5048757	total: 475ms	remaining: 119ms
4:	learn: 0.4744605	total: 593ms	remaining: 0us
catboost:
[[740   1]
 [ 79   0]]
precision: 0.0
recall: 0.0
f1-score: 0.0
accuracy: 0.9024390243902439

cat:
[[740   1]
 [ 79   0]]
precision: 0.0
recall: 0.0
f1-score: 0.0
accuracy: 0.9024390243902439

0:	learn: 0.6541715	total: 1.99s	remaining: 16m 31s
50:	learn: 0.2036221	total: 1m 39s	remaining: 14m 38s
100:	learn: 0.1237313	total: 3m 16s	remaining: 12m 57s
150:	learn: 0.0764438	total: 4m 56s	remaining: 11m 25s
200:	learn: 0.0386639	total: 6m 30s	rema

# Word Embedding

## Word2Vec

In [10]:


import gensim
print("gensim version:", gensim.__version__)

word2vec_path = "./input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin"

# we only load 200k most common words from Google News corpus 
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=200000) 

gensim version: 4.3.1


In [11]:
def get_average_vec(tokens_list, vector, generate_missing=False, k=300):
    """
        Calculate average embedding value of sentence from each word vector
    """
    
    if len(tokens_list)<1:
        return np.zeros(k)
    
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_embeddings(vectors, text, generate_missing=False, k=300):
    """
        create the sentence embedding
    """
    embeddings = text.apply(lambda x: get_average_vec(x, vectors, generate_missing=generate_missing, k=k))
    return list(embeddings)

In [27]:
embeddings_word2vec = get_embeddings(word2vec_model, df_train["lemmatize_text"], k=300)

print("Embedding matrix size", len(embeddings_word2vec), len(embeddings_word2vec[0]))
print("The sentence: \"%s\" got embedding values: " % df_train["lemmatize_text"][0])
#print(embeddings_word2vec[0])

Embedding matrix size 887 300
The sentence: "kill bill siren awesome seriously make week good call saul subreddit would extremely grateful,crazy bastard,hop post something obvious,find scene funny fact kill,bill kill bill say five point palm exploding heart technique sucksbut dude wait two movie see die fall grass cool,jump conclusion application,possible motivation gabriel angle decision surely actually think rick group decision still carry guilt people leave die know would dead grime co think another angle see yet thought,new friend van buren boy,doughno show,seriously stop sing song arizona moon shining,great shot invisible swordsman,goodnight ned,infamous famous guy el guapo famoushe infamous,ha bad,think mean shut,tell u die like dog,watch dead,take moment acknowledge great three amigo saw movie kid absolutely loved watch adult still draw quoting sing along chevy chase steve martin martin short perfect movie see movie mention much recommend anyone see,give downvote,think would ser

In [28]:

embeddings_word2vec_test = get_embeddings(word2vec_model, df_test["lemmatize_text"], k=300)

print("Embedding matrix size", len(embeddings_word2vec), len(embeddings_word2vec[0]))
print("The sentence: \"%s\" got embedding values: " % df_test["lemmatize_text"][0])
#print(embeddings_word2vec[0])

Embedding matrix size 887 300
The sentence: "today learn saiga specie antelope unusually bulbous nose rapidly lose 80 population since may 2015 follow outbreak bacterial disease,bernie sander say find dead body democratic party meeting select candidate decide policy oh god caucus cuz new england accent,trouble faux iosefka yurie pretty difficult still never beat bloody crow,trouble yurie faux iosefka well,bloody crow cainhurst sure count bos fog door,would rank darkbeast paarl,lucky lifetime maybe twice really lucky get opportunity support presidential candidate speaks value deep personal level chance come day bernie sander announce presidential campaign remove,real life,somewhat relate first bloodborne relate dream last night go hunter nightmare buy grocery,pass legal tender illegal tender,crisp fun say word move back mouth front say really like british pronunciation oregano well,united state bmerica,cat,bark,woof woof woof bark,zac volibear,one friend sixth nexus 6p phone available s

In [29]:
# evaluate each model on the training set

for clf_name, clf in get_models().items():
    clf.fit(embeddings_word2vec, y_train)
    y_pred = clf.predict(embeddings_word2vec_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[696  45]
 [ 58  21]]
precision: 0.3181818181818182
recall: 0.26582278481012656
f1-score: 0.28965517241379307
accuracy: 0.874390243902439

XGB:
[[716  25]
 [ 59  20]]
precision: 0.4444444444444444
recall: 0.25316455696202533
f1-score: 0.3225806451612903
accuracy: 0.8975609756097561

0:	learn: 0.6105285	total: 26.3ms	remaining: 105ms
1:	learn: 0.5439029	total: 44.5ms	remaining: 66.8ms
2:	learn: 0.4923819	total: 62.7ms	remaining: 41.8ms
3:	learn: 0.4509138	total: 81.5ms	remaining: 20.4ms
4:	learn: 0.4186706	total: 101ms	remaining: 0us
catboost:
[[733   8]
 [ 70   9]]
precision: 0.5294117647058824
recall: 0.11392405063291139
f1-score: 0.18749999999999997
accuracy: 0.9048780487804878

cat:
[[722  19]
 [ 62  17]]
precision: 0.4722222222222222
recall: 0.21518987341772153
f1-score: 0.29565217391304344
accuracy: 0.901219512195122

0:	learn: 0.6559202	total: 411ms	remaining: 3m 25s
50:	learn: 0.1249140	total: 22.6s	remaining: 3m 19s
100:	learn: 0.0565313	total: 44.8s	remaining: 2m 56s
150

# fastText

In [ ]:

from gensim.models.fasttext import FastText

fasttext_path = "./input/fasttext-wikinews/wiki-news-300d-1M.vec"
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_path, binary=False, limit=200000)

In [12]:

from gensim.models.fasttext import FastText

fasttext_path = "./input/fasttext-wikinews/wiki-news-300d-1M-subword.vec"
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_path, binary=False, limit=200000)

In [12]:

from gensim.models.fasttext import FastText

fasttext_path = "./input/fasttext-wikinews/crawl-300d-2M.vec"
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_path, binary=False, limit=200000)

In [26]:

from gensim.models.fasttext import FastText

fasttext_path = "./input/fasttext-wikinews/crawl-300d-2M-subword.vec"
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_path, binary=False, limit=200000)

In [27]:
embeddings_fasttext = get_embeddings(fasttext_model, df_train["lemmatize_text"], k=300)

print("Embedding matrix size", len(embeddings_fasttext), len(embeddings_fasttext[0]))
print("The sentence: \"%s\" got embedding values: " % df_train["lemmatize_text"][0])
#print(embeddings_fasttext[0])

Embedding matrix size 887 300
The sentence: "kill bill siren awesome seriously make week good call saul subreddit would extremely grateful,crazy bastard,hop post something obvious,find scene funny fact kill,bill kill bill say five point palm exploding heart technique sucksbut dude wait two movie see die fall grass cool,jump conclusion application,possible motivation gabriel angle decision surely actually think rick group decision still carry guilt people leave die know would dead grime co think another angle see yet thought,new friend van buren boy,doughno show,seriously stop sing song arizona moon shining,great shot invisible swordsman,goodnight ned,infamous famous guy el guapo famoushe infamous,ha bad,think mean shut,tell u die like dog,watch dead,take moment acknowledge great three amigo saw movie kid absolutely loved watch adult still draw quoting sing along chevy chase steve martin martin short perfect movie see movie mention much recommend anyone see,give downvote,think would ser

In [28]:
embeddings_fasttext_test = get_embeddings(fasttext_model, df_test["lemmatize_text"], k=300)

print("Embedding matrix size", len(embeddings_fasttext), len(embeddings_fasttext[0]))
print("The sentence: \"%s\" got embedding values: " % df_test["lemmatize_text"][0])
#print(embeddings_fasttext[0])

Embedding matrix size 887 300
The sentence: "today learn saiga specie antelope unusually bulbous nose rapidly lose 80 population since may 2015 follow outbreak bacterial disease,bernie sander say find dead body democratic party meeting select candidate decide policy oh god caucus cuz new england accent,trouble faux iosefka yurie pretty difficult still never beat bloody crow,trouble yurie faux iosefka well,bloody crow cainhurst sure count bos fog door,would rank darkbeast paarl,lucky lifetime maybe twice really lucky get opportunity support presidential candidate speaks value deep personal level chance come day bernie sander announce presidential campaign remove,real life,somewhat relate first bloodborne relate dream last night go hunter nightmare buy grocery,pass legal tender illegal tender,crisp fun say word move back mouth front say really like british pronunciation oregano well,united state bmerica,cat,bark,woof woof woof bark,zac volibear,one friend sixth nexus 6p phone available s

In [29]:
# evaluate each model on the crawl-300d-2M sub word

for clf_name, clf in get_models().items():
    clf.fit(embeddings_fasttext, y_train)
    y_pred = clf.predict(embeddings_fasttext_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[689  52]
 [ 53  26]]
precision: 0.3333333333333333
recall: 0.3291139240506329
f1-score: 0.3312101910828026
accuracy: 0.8719512195121951

XGB:
[[717  24]
 [ 54  25]]
precision: 0.5102040816326531
recall: 0.31645569620253167
f1-score: 0.390625
accuracy: 0.9048780487804878

0:	learn: 0.6062680	total: 26.4ms	remaining: 106ms
1:	learn: 0.5364352	total: 44.6ms	remaining: 66.9ms
2:	learn: 0.4776668	total: 63.8ms	remaining: 42.5ms
3:	learn: 0.4376228	total: 82.5ms	remaining: 20.6ms
4:	learn: 0.3984056	total: 102ms	remaining: 0us
catboost:
[[731  10]
 [ 69  10]]
precision: 0.5
recall: 0.12658227848101267
f1-score: 0.20202020202020204
accuracy: 0.9036585365853659

cat:
[[720  21]
 [ 59  20]]
precision: 0.4878048780487805
recall: 0.25316455696202533
f1-score: 0.33333333333333337
accuracy: 0.9024390243902439

0:	learn: 0.6519109	total: 372ms	remaining: 3m 5s
50:	learn: 0.1165901	total: 20.6s	remaining: 3m 1s
100:	learn: 0.0522246	total: 40.8s	remaining: 2m 41s
150:	learn: 0.0294067	total: 1

In [15]:
# evaluate each model on the crawl-300d-2M

for clf_name, clf in get_models().items():
    clf.fit(embeddings_fasttext, y_train)
    y_pred = clf.predict(embeddings_fasttext_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[692  49]
 [ 51  28]]
precision: 0.36363636363636365
recall: 0.35443037974683544
f1-score: 0.358974358974359
accuracy: 0.8780487804878049

XGB:
[[712  29]
 [ 57  22]]
precision: 0.43137254901960786
recall: 0.27848101265822783
f1-score: 0.3384615384615385
accuracy: 0.8951219512195122

0:	learn: 0.6178201	total: 175ms	remaining: 699ms
1:	learn: 0.5422048	total: 206ms	remaining: 309ms
2:	learn: 0.4856670	total: 228ms	remaining: 152ms
3:	learn: 0.4524312	total: 248ms	remaining: 62ms
4:	learn: 0.4149609	total: 268ms	remaining: 0us
catboost:
[[737   4]
 [ 74   5]]
precision: 0.5555555555555556
recall: 0.06329113924050633
f1-score: 0.11363636363636365
accuracy: 0.9048780487804878

cat:
[[721  20]
 [ 55  24]]
precision: 0.5454545454545454
recall: 0.3037974683544304
f1-score: 0.39024390243902435
accuracy: 0.9085365853658537

0:	learn: 0.6482775	total: 368ms	remaining: 3m 3s
50:	learn: 0.1198171	total: 21s	remaining: 3m 4s
100:	learn: 0.0530198	total: 41.8s	remaining: 2m 45s
150:	learn: 0.

In [15]:
# evaluate each model on the wiki-news-300d-1M-subword

for clf_name, clf in get_models().items():
    clf.fit(embeddings_fasttext, y_train)
    y_pred = clf.predict(embeddings_fasttext_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[702  39]
 [ 50  29]]
precision: 0.4264705882352941
recall: 0.3670886075949367
f1-score: 0.3945578231292517
accuracy: 0.8914634146341464

XGB:
[[719  22]
 [ 57  22]]
precision: 0.5
recall: 0.27848101265822783
f1-score: 0.35772357723577236
accuracy: 0.9036585365853659

0:	learn: 0.6095910	total: 177ms	remaining: 708ms
1:	learn: 0.5421078	total: 197ms	remaining: 295ms
2:	learn: 0.4868791	total: 217ms	remaining: 145ms
3:	learn: 0.4460389	total: 237ms	remaining: 59.2ms
4:	learn: 0.4101548	total: 257ms	remaining: 0us
catboost:
[[734   7]
 [ 74   5]]
precision: 0.4166666666666667
recall: 0.06329113924050633
f1-score: 0.1098901098901099
accuracy: 0.901219512195122

cat:
[[724  17]
 [ 55  24]]
precision: 0.5853658536585366
recall: 0.3037974683544304
f1-score: 0.4
accuracy: 0.9121951219512195

0:	learn: 0.6470351	total: 385ms	remaining: 3m 11s
50:	learn: 0.1209974	total: 21s	remaining: 3m 4s
100:	learn: 0.0540901	total: 41.5s	remaining: 2m 44s
150:	learn: 0.0309443	total: 1m 2s	remaining:

In [ ]:
# evaluate each model on the wiki-news-300d-1M

for clf_name, clf in get_models().items():
    clf.fit(embeddings_fasttext, y_train)
    y_pred = clf.predict(embeddings_fasttext_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[700  41]
 [ 56  23]]
precision: 0.359375
recall: 0.2911392405063291
f1-score: 0.32167832167832167
accuracy: 0.8817073170731707

XGB:
[[715  26]
 [ 58  21]]
precision: 0.44680851063829785
recall: 0.26582278481012656
f1-score: 0.3333333333333333
accuracy: 0.8975609756097561

0:	learn: 0.6206111	total: 30.9ms	remaining: 124ms
1:	learn: 0.5422569	total: 53.2ms	remaining: 79.8ms
2:	learn: 0.4884149	total: 74ms	remaining: 49.3ms
3:	learn: 0.4542586	total: 96.9ms	remaining: 24.2ms
4:	learn: 0.4128798	total: 124ms	remaining: 0us
catboost:
[[734   7]
 [ 72   7]]
precision: 0.5
recall: 0.08860759493670886
f1-score: 0.15053763440860216
accuracy: 0.9036585365853659

cat:
[[725  16]
 [ 58  21]]
precision: 0.5675675675675675
recall: 0.26582278481012656
f1-score: 0.3620689655172413
accuracy: 0.9097560975609756

0:	learn: 0.6508347	total: 425ms	remaining: 3m 32s
50:	learn: 0.1185822	total: 24.8s	remaining: 3m 38s
100:	learn: 0.0530064	total: 48.1s	remaining: 3m 10s
150:	learn: 0.0300549	total: 

In [35]:
embeddings_fasttext = get_embeddings(fasttext_model, df_train_wo["lemmatize_text_wo"], k=300)

print("Embedding matrix size", len(embeddings_fasttext), len(embeddings_fasttext[0]))
print("The sentence: \"%s\" got embedding values: " % df_train_wo["lemmatize_text_wo"][0])
#print(embeddings_fasttext[0])

Embedding matrix size 887 300
The sentence: "kill bill siren awesome seriously make week better call saul subreddit would extremely grateful,crazy bastard,hoped post something obvious,find scene funny fact kill,bill kill bill saying five point palm exploding heart technique sucksbut dude waited two movie see die fall grass cool,jump conclusion application,possible motivation gabriel angle decision surely actually thinking rick group decision still carry guilt people left die know would dead grime co thinking another angle seeing yet thought,new friend van buren boy,doughno show,seriously stop singing song arizona moon shining,great shot invisible swordsman,goodnight ned,infamous famous guy el guapo famoushe infamous,ha bad,think mean shut,tell u die like dog,watch dead,take moment acknowledge great three amigo saw movie kid absolutely loved watched adult still drawn quoting singing along chevy chase steve martin martin short perfect movie see movie mentioned much recommend anyone seen,

In [36]:
embeddings_fasttext_test = get_embeddings(fasttext_model, df_test_wo["lemmatize_text_wo"], k=300)

print("Embedding matrix size", len(embeddings_fasttext), len(embeddings_fasttext[0]))
print("The sentence: \"%s\" got embedding values: " % df_test_wo["lemmatize_text_wo"][0])
#print(embeddings_fasttext[0])

Embedding matrix size 887 300
The sentence: "today learned saiga specie antelope unusually bulbous nose rapidly lost 80 population since may 2015 following outbreak bacterial disease,bernie sander say found dead body democratic party meeting select candidate decide policy oh god caucus cuz new england accent,trouble faux iosefka yurie pretty difficult still never beat bloody crow,trouble yurie faux iosefka well,bloody crow cainhurst sure count bos fog door,would rank darkbeast paarl,lucky lifetime maybe twice really lucky get opportunity support presidential candidate speaks value deep personal level chance came day bernie sander announced presidential campaign removed,real life,somewhat related first bloodborne related dream last night went hunter nightmare buy grocery,passed legal tender illegal tender,crisp fun say word move back mouth front say really like british pronunciation oregano well,united state bmerica,cat,bark,woof woof woof bark,zac volibear,one friend sixth nexus 6p pho

In [37]:
# evaluate each model on the training set

for clf_name, clf in get_models().items():
    clf.fit(embeddings_fasttext, y_train)
    y_pred = clf.predict(embeddings_fasttext_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()

Ada:
[[701  40]
 [ 53  26]]
precision: 0.3939393939393939
recall: 0.3291139240506329
f1-score: 0.3586206896551724
accuracy: 0.8865853658536585

XGB:
[[720  21]
 [ 53  26]]
precision: 0.5531914893617021
recall: 0.3291139240506329
f1-score: 0.4126984126984127
accuracy: 0.9097560975609756

0:	learn: 0.6058939	total: 27ms	remaining: 108ms
1:	learn: 0.5446190	total: 45.6ms	remaining: 68.3ms
2:	learn: 0.4853507	total: 63.7ms	remaining: 42.5ms
3:	learn: 0.4524286	total: 82.9ms	remaining: 20.7ms
4:	learn: 0.4149303	total: 105ms	remaining: 0us
catboost:
[[736   5]
 [ 75   4]]
precision: 0.4444444444444444
recall: 0.05063291139240506
f1-score: 0.0909090909090909
accuracy: 0.9024390243902439

cat:
[[721  20]
 [ 53  26]]
precision: 0.5652173913043478
recall: 0.3291139240506329
f1-score: 0.416
accuracy: 0.9109756097560976

0:	learn: 0.6506714	total: 428ms	remaining: 3m 33s
50:	learn: 0.1190206	total: 22.9s	remaining: 3m 21s
100:	learn: 0.0530458	total: 45.5s	remaining: 2m 59s
150:	learn: 0.0297893	

# Glove

In [16]:
%time 
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim import models


'''glove_input_file = "./input/glove6b/glove.6B.300d.txt"
word2vec_output_file = "glove.6B.100d.txt.word2vec"
'''
glove_input_file = "./input/glove27b/glove.twitter.27B.200d.txt"
word2vec_output_file = "glove.twwiter.27B.200d.txt.word2vec"
glove2word2vec(glove_input_file, word2vec_output_file)

# we only load 200k most common words from Google New corpus 
glove_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False, limit=200000) 




CPU times: total: 0 ns
Wall time: 0 ns


In [20]:
%time 
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim import models


glove_input_file = "./input/glove6b/glove.6B.300d.txt"
word2vec_output_file = "glove.6B.100d.txt.word2vec"


glove2word2vec(glove_input_file, word2vec_output_file)

# we only load 200k most common words from Google New corpus 
glove_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False, limit=200000) 




CPU times: total: 0 ns
Wall time: 0 ns


In [23]:
embeddings_glove = get_embeddings(glove_model, df_train["lemmatize_text"], k=300)

print("Embedding matrix size", len(embeddings_glove), len(embeddings_glove[0]))
print("The sentence: \"%s\" got embedding values: " % df_train["lemmatize_text"][0])
#print(embeddings_glove[0])


Embedding matrix size 887 300
The sentence: "kill bill siren awesome seriously make week good call saul subreddit would extremely grateful,crazy bastard,hop post something obvious,find scene funny fact kill,bill kill bill say five point palm exploding heart technique sucksbut dude wait two movie see die fall grass cool,jump conclusion application,possible motivation gabriel angle decision surely actually think rick group decision still carry guilt people leave die know would dead grime co think another angle see yet thought,new friend van buren boy,doughno show,seriously stop sing song arizona moon shining,great shot invisible swordsman,goodnight ned,infamous famous guy el guapo famoushe infamous,ha bad,think mean shut,tell u die like dog,watch dead,take moment acknowledge great three amigo saw movie kid absolutely loved watch adult still draw quoting sing along chevy chase steve martin martin short perfect movie see movie mention much recommend anyone see,give downvote,think would ser

In [24]:

embeddings_glove_test = get_embeddings(glove_model, df_test["lemmatize_text"], k=300)

print("Embedding matrix size", len(embeddings_glove), len(embeddings_glove[0]))
print("The sentence: \"%s\" got embedding values: " % df_test["lemmatize_text"][0])
#print(embeddings_glove[0])



Embedding matrix size 887 300
The sentence: "today learn saiga specie antelope unusually bulbous nose rapidly lose 80 population since may 2015 follow outbreak bacterial disease,bernie sander say find dead body democratic party meeting select candidate decide policy oh god caucus cuz new england accent,trouble faux iosefka yurie pretty difficult still never beat bloody crow,trouble yurie faux iosefka well,bloody crow cainhurst sure count bos fog door,would rank darkbeast paarl,lucky lifetime maybe twice really lucky get opportunity support presidential candidate speaks value deep personal level chance come day bernie sander announce presidential campaign remove,real life,somewhat relate first bloodborne relate dream last night go hunter nightmare buy grocery,pass legal tender illegal tender,crisp fun say word move back mouth front say really like british pronunciation oregano well,united state bmerica,cat,bark,woof woof woof bark,zac volibear,one friend sixth nexus 6p phone available s

In [25]:
# evaluate each model on the   glove.6B.300d

for clf_name, clf in get_models().items():
    clf.fit(embeddings_glove, y_train)
    y_pred = clf.predict(embeddings_glove_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()


Ada:
[[705  36]
 [ 53  26]]
precision: 0.41935483870967744
recall: 0.3291139240506329
f1-score: 0.36879432624113473
accuracy: 0.8914634146341464

XGB:
[[714  27]
 [ 57  22]]
precision: 0.4489795918367347
recall: 0.27848101265822783
f1-score: 0.34375
accuracy: 0.8975609756097561

0:	learn: 0.6040971	total: 27.2ms	remaining: 109ms
1:	learn: 0.5244657	total: 45.3ms	remaining: 68ms
2:	learn: 0.4750419	total: 63.8ms	remaining: 42.5ms
3:	learn: 0.4362656	total: 83.4ms	remaining: 20.9ms
4:	learn: 0.4011550	total: 103ms	remaining: 0us
catboost:
[[732   9]
 [ 68  11]]
precision: 0.55
recall: 0.13924050632911392
f1-score: 0.22222222222222224
accuracy: 0.9060975609756098

cat:
[[725  16]
 [ 55  24]]
precision: 0.6
recall: 0.3037974683544304
f1-score: 0.40336134453781514
accuracy: 0.9134146341463415

0:	learn: 0.6463146	total: 354ms	remaining: 2m 56s
50:	learn: 0.1204808	total: 20.7s	remaining: 3m 2s
100:	learn: 0.0537753	total: 41.6s	remaining: 2m 44s
150:	learn: 0.0301979	total: 1m 2s	remaining:

In [19]:
# evaluate each model on the glove.twitter.27B.200d

for clf_name, clf in get_models().items():
    clf.fit(embeddings_glove, y_train)
    y_pred = clf.predict(embeddings_glove_test)
    conf = eval_confusion(y_pred, y_test)
    print("{}:".format(clf_name))
    print(conf['matrix'])
    print("precision: {}".format(conf['precision']))
    print("recall: {}".format(conf['recall']))
    print("f1-score: {}".format(conf['f1']))
    print("accuracy: {}".format(accuracy_score(y_pred, y_test)))
    print()


Ada:
[[710  31]
 [ 56  23]]
precision: 0.42592592592592593
recall: 0.2911392405063291
f1-score: 0.3458646616541353
accuracy: 0.8939024390243903

XGB:
[[719  22]
 [ 56  23]]
precision: 0.5111111111111111
recall: 0.2911392405063291
f1-score: 0.3709677419354839
accuracy: 0.9048780487804878

0:	learn: 0.6193173	total: 12.1ms	remaining: 48.4ms
1:	learn: 0.5526841	total: 24.2ms	remaining: 36.3ms
2:	learn: 0.4989086	total: 39.4ms	remaining: 26.2ms
3:	learn: 0.4473470	total: 52.4ms	remaining: 13.1ms
4:	learn: 0.4146409	total: 64.2ms	remaining: 0us
catboost:
[[731  10]
 [ 74   5]]
precision: 0.3333333333333333
recall: 0.06329113924050633
f1-score: 0.10638297872340426
accuracy: 0.8975609756097561

cat:
[[726  15]
 [ 57  22]]
precision: 0.5945945945945946
recall: 0.27848101265822783
f1-score: 0.37931034482758613
accuracy: 0.9121951219512195

0:	learn: 0.6452542	total: 252ms	remaining: 2m 5s
50:	learn: 0.1222826	total: 14.6s	remaining: 2m 8s
100:	learn: 0.0551787	total: 29.5s	remaining: 1m 56s
150